CS524: Introduction to Optimization Lecture 17
======================================

#### Michael C. Ferris<br>Computer Sciences Department<br>University of Wisconsin-Madison
#### October 13, 2023

## Discrete variables

- In many modeling situations some or all of the variables are constrained to
 take on only integer values.

- In addition to ``positive`` and ``free variables``, GAMS has ``integer variables`` $x \in \{ 0,1,2,\ldots\}$ and ``binary variables`` $x \in \{ 0, 1 \}$.

- Any model that involves discrete variables (and otherwise) linear constraints is a ``mip`` and not a linear program.  The solve statement must be modified:

 solve modname using mip minimizing obj;

- Such problems are much harder to solve as we now see.  
- Note that replacing ``mip`` by ``rmip`` in the above solve statement relaxes the integer constraints so the variable are continuous and the resulting model is then a linear program (but is a ``relaxation``).
  

## Semiint, semicont or sos (other discrete) variables}


- Note that GAMS can use semiint or semicontinuous variables that are also discrete variables. See the ``Variables'' section of the user guide.
- Semicontinuous is a variable that is either zero, or else in a given range $[L,U]$. Define ``semicont variable x; x.lo = L; x.up = U;``
- Semiint is the same, except that there is also an integer restriction. It's a variable that is either zero or else an integer in
the range $[L,U]$.
- Not all mip solvers allow these variable types so we often just model using  binary and integer variables.
- Example: ``dorian2.gms``
- There are also ``sos1`` and ``sos2`` variables} as we shall use later.

## Integer Programs are hard!
<ul>
    <li>IP models can be very much more difficult to solve than LP models.</li>
    <li><b>I’m not kidding<br>
           For example, an IP model can take 2 years of CPU time to solve. </b></li>
    <li>It is important that you have a handle on...
        <ol>
            <li>How to build a problem that is likely to be solved – <b>Proper formulation is important!</b> This can affect how long the program will run. </li>
            <li>The general ideas of how integer programming problems are solved: <font color="red">Branch and Bound.</font></li>
        </ol>
    </li>
</ul>

## It’s All About Relaxations
<img src="ST.png" style="height: 300px;" class="center"/>
Here, the red rectangle $ S $ can be seen as the region holding the solutions of the IP problem, while the green hexagon $ T $ is the region holding the solutions of the relaxed LP problem.<br>

The optimal solution in each region can be written as
<ul>
    <li>Let $ z_S = \max f(x):x \in S$</li>
    <li>Let $ z_T = \max f(x):x \in T$</li>
</ul>
<br>

## Obvious, but Important Stuff

$$ z_T \geq z_S \text{(Always true!!!)}$$

<br>
<ul>
    <li>If $ x^*_T $ is an optimal solution to $ \max f(x):x \in T $</li>
<li>And $ x^*_T \in S$, then</li>
<li>$ x^*_T $ is an optimal solution to $ \max f(x):x \in S $</li>
<li>If we were to replace $ \max $ by $ \min $, then $ z_T \leq z_S $. <font color="red">Everything is just flipped!</font></li>
</ul>

## LP relaxation
Consider the IP problem<br><br>

maximize
$$ z_{IP} = c^T x $$
subject to
$$ Ax \geq b $$
$$ x \geq 0 $$
$$ x_j \in \mathbb{Z}, j \in P \subseteq N $$

This is essentially the "red rectangle" problem.

## LP relaxation
This IP problem can be relaxed to the LP problem <br><br>

maximize
$$ z_{LP} = c^T x $$
subject to
$$ Ax \geq b $$
$$ x \geq 0 $$

This is essentially the "green hexagon" problem.

<ul>
    <li>$ z_{LP} \geq z_{IP}$<br></li>
    <li>If $ x^* $ solves the LP relaxation and $ x^* $ satisfies the integrality requirements ($ x^* \in S $), then $ x^* $ solves IP.</li>
</ul>

## $ x^* \notin S $?
<ul>
    <li>Then we <font color="red">branch!</font></li>
    <li>Partition the problem into smaller pieces.</li>
    <li>$ x^* \notin S \Rightarrow \exists k \in P $ such that $ x^*_k $ is fractional.</li>
    <li>Create two new (restricted IP) problems...</li>
</ul>
<ol>
    <li>In one problem, add the constraint $ x_k \leq \lfloor{x^*_k}\rfloor $</li>
    <li>In the other problem, add the constraint $ x_k \geq \lceil{x^*_k}\rceil $</li>
</ol>

## What Does All That Fancy Notation Mean?
Consider the IP problem<br><br>
maximize
$$ z = 5x_1 + 4x_2 $$
subject to
$$ x_1 + x_2 \leq 5 $$
$$ 10x_1 + 6x_2 \leq 45 $$
$$ x_1, x_2 \geq 0 $$
$$ x_1,x_2 \in \mathbb{Z} \text{(must be integer valued)} $$

## The Feasible Region
<img src="FeasibleRegion.png" style="height: 300px;" class="center"/>
Here, the "red dots" are the feasible region (i.e. integers). <br>

## The Feasible Region
<img src="FeasibleRegion.png" style="height: 300px;" class="center"/>
The optimal solution of the LP relaxation of this problem is $ z = 23.75 $, $ x_1 = 3.75 $, $ x_2 = 1.25 $.<br>
However, these solution values do not satisfy the integral restrictions.

## Branching
To solve this problem, divide (or "branch") the problem into two smaller restricted IP problems, where one has $ x_1 \leq 3 $ and the other has $ x_1 \geq 4 $, as shown: <br>

<img src="Branching.png" style="height: 300px;" class="center"/>

Now, the IP solution can be found by recursively solving the LP relaxations of the two smaller restricted IP problems.<br>

## The Branch and Bound Algorithm
<ul>
    <li>All of the following assumes a <i>maximization</i> problem.</li>
    <li>It works equally well for minimization, but you need to replace “lower” by “upper” (and vice verse) everywhere.</li>
    <li>Let $ z_L $ be a lower bound on optimal objective value. Originally $ z_L = -\infty $. The initial list of LP relaxations to solve consists of just the LP relaxation to the entire problem.</li>
</ul>

## Branch and Bound (1 of 2)

<ol>
    <li>Select a restricted IP problem.</li>
    <li>Solve the LP relaxation of the restricted IP problem. If the LP relaxation is infeasible, Go to 1.</li>
    <li>Otherwise, let $ x^* $ be the optimal solution to the LP relaxation and let $ z^* $ be its objective value.</li>
    <li>If $ x^* $ satisfies the integer restrictions, then $ z^* $ is the optimal value for the restricted IP problem. If $ z^* \gt z_L $, then $ z_L := z^* $. (Also keep track of $ x^* $ as a candidate solution). Go to 1.</li>
</ol>

## Branch and Bound (2 of 2)

<ol start="5">
    <li>Otherwise, $ x^* $ does not satisfy the integer restrictions. $ z^* $ is an upper bound on the optimal value of the restricted IP problem.</li>
    <li>If $ z^* \leq z_L $, (<font color="red">Fathom</font>) Go to 1. (There is no way that a better optimal solution can be in this restricted region).</li>
    <li>Otherwise, <font color="red">Divide</font>. Choose some index $ k $ such that $ x^*_k $ does not satsify the integer restriction. Add two new restricted problems to the list:
        <ol>
            <li>Include constraint $ x_k \leq \lfloor{x^*_k}\rfloor $</li>
            <li>Include constraint $ x_k \geq \lceil{x^*_k}\rceil $</li>
        </ol></li>
    <li>Go to 1.</li>
</ol>

## Simple knapsack example
$$ \max_x{8x_1 + 6x_2 + 5x_3 + 4x_4} $$
$$ 5x_1 + 4x_2 + 4x_3 + 2x_4 \leq 8 $$
$$ x_1, x_2, x_3, x_4 \in \{0, 1\} $$

In [1]:
%load_ext gams.magic
m = gams.exchange_container
x1 = m.addVariable('x1','binary')
x2 = m.addVariable('x2','binary')
x3 = m.addVariable('x3','binary')
x4 = m.addVariable('x4','binary')

In [2]:
%%gams
variable obj_var;

Equations eq, obj;
obj..    8*x1 + 6*x2 + 5*x3 + 4*x4 =e= obj_var;
eq..     5*x1 + 4*x2 + 4*x3 + 2*x4 =l= 8;  

model bb/all/;

# b_and_b.gms

The lower and upper bounds are initialized to be
$$ z_L = -\infty, z_U = \infty $$
First solve the basic LP relaxation of the original IP problem (restricted IP problem).<br>

In [3]:
gams.gams('solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),13.5,2,5,RMIP,CPLEX,0.015


x = (1.0, 0.25, 0.0, 1.0)


# b_and_b.gms
The current LP relaxation is feasible, but $ x_2 $ is not an integer. <br>
<img src="BranchandBoundProcess1.png" style="height: 300px;" class="center"/><br>
Here, $ z = 13.5 $ is the upper bound for all of the remaining restricted problems. Thus, the current bounds become
$$ z_L = -\infty, z_U = 13.5 $$

# b_and_b.gms
Divide the current problem into two restricted problems, where one has $ x_2 = 0 $ and the other has $ x_2 = 1 $.<br>
<img src="BranchandBoundProcess2.png" style="height: 300px;" class="center"/><br>
First, solve the LP relaxation where $ x_2 = 0 $.

In [4]:
gams.gams('x2.fx=0; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),13.25,2,5,RMIP,CPLEX,0.003


x = (1.0, 0.0, 0.25, 1.0)


# b_and_b.gms
The current LP relaxation is feasible, but $ x_3 $ is not an integer. <br>
Current bounds are
$$ z_L = -\infty, z_U = 13.5 $$

# b_and_b.gms
Divide the current problem into two restricted problems, where one has $ x_2 = 0, x_3 = 0 $ and the other has $ x_2 = 0, x_3 = 1 $.<br>
<img src="BranchandBoundProcess3.png" style="height: 300px;" class="center"/><br>
First, solve the LP relaxation where $ x_2 = 0, x_3 = 0 $.

In [5]:
gams.gams('x3.fx=0; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),12.0,2,5,RMIP,CPLEX,0.001


x = (1.0, 0.0, 0.0, 1.0)


# b_and_b.gms
The current LP relaxation is feasible and all $ x_i $ satisfy the integer restrictions. <br>
<img src="BranchandBoundProcess4.png" style="height: 300px;" class="center"/><br>
Thus, update the lower bound with $ z = 12.0 $. The current bounds become
$$ z_L = 12.0, z_U = 13.5 $$

# b_and_b.gms
Next, solve the LP relaxation where $ x_2 = 0, x_3 = 1 $.

In [6]:
gams.gams('x3.fx=1; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),12.2,2,5,RMIP,CPLEX,0.001


x = (0.4, 0.0, 1.0, 1.0)


# b_and_b.gms
The current LP relaxation is feasible, but $ x_1 $ is not an integer. <br>
<img src="BranchandBoundProcess5.png" style="height: 300px;" class="center"/><br>
Current bounds are
$$ z_L = 12.0, z_U = 13.5 $$

# b_and_b.gms
By rounding the objective value from $ z = 12.2 $ to $ z = 12.0 $ (this can be done since $ z $ must be an integer), it can be seen that
$$ z \leq z_L $$
Thus, the current problem can be fathomed, since there is no better optimal solution in this restricted region.
<img src="BranchandBoundProcess6.png" style="height: 300px;" class="center"/>

# b_and_b.gms
Next, go back to the very first "branch" and solve the LP relaxation where $ x_2 = 1 $.

In [7]:
gams.gams('x2.fx=1; x3.lo=0; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),13.2,2,5,RMIP,CPLEX,0.001


x = (0.4, 1.0, 0.0, 1.0)


# b_and_b.gms
The current LP relaxation is feasible, but $ x_1 $ is not an integer. <br>
Here, $ z = 13.2 $ is the upper bound for all of the remaining restricted problems. Thus, the current bounds become
$$ z_L = 12.0, z_U = 13.2 $$

# b_and_b.gms
Divide the current problem into two restricted problems, where one has $ x_2 = 1, x_1 = 0 $ and the other has $ x_2 = 1, x_1 = 1 $.<br>
<img src="BranchandBoundProcess7.png" style="height: 300px;" class="center"/><br>
First, solve the LP relaxation where $ x_2 = 1, x_1 = 0 $.

In [8]:
gams.gams('x1.fx=0; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),12.5,2,5,RMIP,CPLEX,0.001


x = (0.0, 1.0, 0.5, 1.0)


# b_and_b.gms
The current LP relaxation is feasible, but $ x_3 $ is not an integer. <br>
<img src="BranchandBoundProcess8.png" style="height: 300px;" class="center"/>
Current bounds are
$$ z_L = 12.0, z_U = 13.2 $$

# b_and_b.gms
By rounding the objective value (which in this instance must be integral) from $ z = 12.5 $ to $ z = 12.0 $, it can be seen that
$$ z \leq z_L $$
Thus, the current problem can be fathomed.
<img src="BranchandBoundProcess9.png" style="height: 300px;" class="center"/>

# b_and_b.gms
Next, solve the LP relaxation where $ x_2 = 1, x_1 = 1 $.

In [9]:
gams.gams('x1.fx=1; solve bb using rmip maximizing obj_var;')
print(f'x = ({x1.toValue()}, {x2.toValue()}, {x3.toValue()}, {x4.toValue()})')

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),InfeasibleGlobal (4),NA,2,5,RMIP,CPLEX,0


x = (1.0, 1.0, 0.5, 1.0)


# b_and_b.gms
The current LP relaxation is infeasible. <br>
<img src="BranchandBoundProcess10.png" style="height: 300px;" class="center"/>

# b_and_b.gms
Since all restricted IP problems have been solved, the bounds can be updated to
$$ z_L = 12.0, z_U = 12.0 $$
The optimal solution to this IP problem is thus
$$ z^* = 12.0 $$
$$ x^*_1 = 1, x^*_2 = 0, x^*_3 = 0, x^*_4 = 1 $$

In [10]:
gams.gams_cleanup('--closedown')

## Bounds and performance guarantees

<ul>
<li> We saw in the description of the algorithm how to get a lower
bound $z_L$ on the optimal objective function value</li>
<li> We can also get an upper bound on the optimal objective
value.  $z_U$ is the maximum of all of the upper bounds of all of the
remaining restriced problems that must be evaluated</li>
<li> If we only want to solve a problem to within $\alpha$% of
optimality, we can!!!</li>
<li> This is really what is great about optimization.  Not only do
you get a solution, but you can tell your boss that if there is a
better solution it can be at most $\alpha$% better.</li>
</ul>

## GAMS and Tolerances
<ul>
    <li><b><font face = "Courier New">option optcr = 0.0</font></b></li>
    <li><b><font face = "Courier New">option optca = 0.0</font></b></li>
</ul>
<ul>
    <li><b><font face = "Courier New">optcr</font></b>: Stop if $ (z_U - z_L) / \max{\{|z_L|, 1\}} \leq $ <b><font face = "Courier New">optcr</font></b></li>
    <li><b><font face = "Courier New">optca</font></b>: Stop if $ (z_U - z_L) \leq $ <b><font face = "Courier New">optca</font></b></li>
</ul>

## Some Good CPLEX parameters
<ul>
    <li>mipemphasis (1 for solution finding, 2 for optimality)</li>
    <li>varsel (strong branching 3)</li>
    <li>cuts (-1 no, 5 aggressive)</li>
    <li>probe (-1 no, 3 full)</li>
    <li>lbheur (local branching heuristic)</li>
    <li>preslvnd</li>
    <li>repeatepresolve</li>
    <li>subalg</li>
    <li>nodesel (0 dfs, 2 best estimate)</li>
    <li>heurfreq (-1 for node heuristic off)</li>
    <li>mipstart 1 (use an existing integer solution to start)</li>
    <li>numericalemphasis (1 for extreme caution)</li>
</ul>

## Some Good Gurobi parameters
<ul>
    <li>cuts -1: auto, 0:off, 3:aggresive</li>
    <li>heuristics (fraction) % of time spent trying to find feasible solutions</li>
    <li>mipfocus: 1 for solution finding, 2 for optimality</li>
    <li>mipstart 1</li>
</ul>

## Additional Materials
<ul>
    <li><a href="https://support.gams.com/solver:what_is_optca_optcr">Gams Tolerances</a></li>
    <li><a href="https://www.gams.com/latest/docs/S_CPLEX.html">CPLEX Solver and Parameters</a></li>
    <li><a href="https://www.gams.com/latest/docs/S_GUROBI.html">Gurobi Solver and Paramters</a></li>
    <li><a href="https://web.stanford.edu/class/ee364b/lectures/bb_slides.pdf">Stanford Lecture on Branch and Bound Methods</a></li>
    <li><a href="https://towardsdatascience.com/the-branch-and-bound-algorithm-a7ae4d227a69">Simple Explanation on Branch and Bound Algorithm</a></li>
</ul>

## Overview
<ul>
    <li>This lecture introduces the basic concept of solving IP models using the Branch and Bound method.</li>
    <li>Briefly, the Branch and Bound method recursively divides and solves the LP relaxations of restricted IP problems.</li>
    <li>During the recursion, a lower and upper bound of the current optimal value is recorded and updated; these bounds are used to trim down the search space of restricted IP problems.</li>
    <li>After all the restricted IP problems have been solved, the optimal solution within the bounds can be obtained.
</ul>